In [ ]:
%tensorflow_version 1.x

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

from google.colab import drive
from math import ceil
import matplotlib.pyplot as plt
import datetime
import numpy as np
import os

# Experiment configuration
USING_SPLITS = False
NET = 'inceptionresnetv2'  # Must be one of the available networks below
current_folder = '/content/gdrive/My Drive/public/sediments/'

samples = 'splits' if USING_SPLITS else 'images'
drive.mount('/content/gdrive')
!cp "{current_folder}{samples}.tar.gz" /tmp
!tar xzvf /tmp/{samples}.tar.gz
BASE = '/content/{}/folds'.format(samples)
ALL_TRAIN_SET_PATH = '/content/splits/train' if USING_SPLITS else '/content/images/training_set'
TEST_SET_PATH = '/content/splits/test' if USING_SPLITS else '/content/images/test_set'
BASE_RESULTS = current_folder + 'results/' + samples

available_networks = {
    'xception': Xception,
    'vgg16': VGG16,
    'vgg19': VGG19,
    'resnet50': ResNet50,
    'resnet50v2': ResNet50V2,
    'inceptionv3': InceptionV3,
    'inceptionresnetv2': InceptionResNetV2,
    'mobilenet': MobileNet,
    'densenet': DenseNet121,
    'nasnetmobile': NASNetMobile,
    'mobilenetv2': MobileNetV2
}

CLASSES = ['background', 'chambers', 'channels', 'packing voids', 'planes', 'vesicles', 'vughs']


def plot_history(histories, base_results, network, fold, additional_suffix_str, key='acc'):
    plt.figure(figsize=(16,10))
  
    val_key = 'val_' + key
    for name, history in histories:
        if val_key in history.history:
            val = plt.plot(history.epoch, history.history[val_key], '--', label=name.title() + ' Val')
            plt.plot(history.epoch, history.history[key], color=val[0].get_color(), label=name.title() + ' Train')
        else:
            plt.plot(history.epoch, history.history[key], color='b', label=name.title() + ' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_', ' ').title())
    plt.legend()
    plt.xlim([0, max(history.epoch)])
    plt.savefig('{0}/{1}/fold_{2}_{3}_acc_history.png'.format(base_results, network, fold, additional_suffix_str),
                dpi=None, facecolor='w', edgecolor='w',  orientation='portrait', papertype=None,
                format=None, transparent=False, bbox_inches='tight', pad_inches=0.5, frameon=None)


def build_model(network, input_size):
    base_model = available_networks[network](input_tensor=Input(shape=(input_size, input_size, 3)),
                                             weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(7, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return base_model, model


def model_fine_tuning(network, base_model, model, train_batches, val_batches, EPOCHS_FINE_TUNING,
                      STEPS_PER_EPOCH, VALIDATION_STEPS, base_results, fold):
    print('Trainable weights before freezing the conv base: {}'.format(len(model.trainable_weights)))
    base_model.trainable = False
    print('Trainable weights after freezing the conv base: {}'.format(len(model.trainable_weights)))
    model.compile(RMSprop(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    print('Epochs: {}'.format(EPOCHS_FINE_TUNING))
    print('Steps per epoch: {}'.format(STEPS_PER_EPOCH))
    print('Validation steps: {}'.format(VALIDATION_STEPS))
    print('Fold {} - Starting fine-tuning...'.format(fold))
    a = datetime.datetime.now()
    model_history = model.fit_generator(train_batches, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS_FINE_TUNING,
                                        verbose=2) if val_batches is None else \
                    model.fit_generator(train_batches, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS_FINE_TUNING,
                                        validation_data=val_batches, validation_steps=VALIDATION_STEPS, verbose=2)
    b = datetime.datetime.now()
    print('Elapsed time for fine-tuning: {}'.format(b - a))
    plot_history([('model', model_history)], base_results, network, fold, 'fine_tuning')
    return model_history, base_model, model

  
def model_deep_tuning(network, input_size, base_model, model, train_batches, val_batches, test_path,
                      EPOCHS_DEEP_TUNING, STEPS_PER_EPOCH, VALIDATION_STEPS, base_results, fold, results_file):
    print('Trainable weights before unfreezing the conv base: {}'.format(len(model.trainable_weights)))
    base_model.trainable = True
    print('Trainable weights after unfreezing the conv base: {}'.format(len(model.trainable_weights)))
    model.compile(RMSprop(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    print('Fold {} - Starting deep-tuning...'.format(fold))
    a = datetime.datetime.now()
    if val_batches is None:
        model_history = model.fit_generator(train_batches, steps_per_epoch=STEPS_PER_EPOCH,
                                            epochs=EPOCHS_DEEP_TUNING, verbose=2)
        model.save_weights('{0}/{1}/fold_{2}_w_net.h5'.format(base_results, network, fold))
        test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
            test_path, target_size=(input_size, input_size), batch_size=BATCH_SIZE, classes=CLASSES)
        loss, acc = model.evaluate_generator(test_batches)
        results_file.write('Fold {0} - Test Acc: {1}\n'.format(fold, acc))
    else:
        model_history = model.fit_generator(train_batches, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS_DEEP_TUNING,
                                            validation_data=val_batches, validation_steps=VALIDATION_STEPS, verbose=2)
        loss, acc = model.evaluate_generator(val_batches)
        results_file.write('Fold {0} - Test Acc: {1}\n'.format(fold, acc))
    b = datetime.datetime.now()
    print('Elapsed time for deep-tuning: {}'.format(b - a))
    plot_history([('model', model_history)], base_results, network, fold, 'deep_tuning')
    results_file.close()
    return model_history


results_path = BASE_RESULTS + '/' + NET
os.makedirs(results_path, exist_ok=True)

# Network parameters
NUM_FOLDS = 1
input_size = 224
BATCH_SIZE = 32
EPOCHS_FINE_TUNING = 30
EPOCHS_DEEP_TUNING = 50 
all_acc_histories = []

# For loop repeating the training for each fold: fine-tuning freezing the base model
for fold in range(1, NUM_FOLDS + 1):
    fold_path = BASE + '/fold_' + str(fold)
    train_path = fold_path + '/training_set'
    val_path = fold_path + '/validation_set'
    results_file = open(results_path + '/info.txt', 'w' if fold == 1 else 'a') 

    base_model, model = build_model(NET, input_size)
    train_generator = ImageDataGenerator(
        rescale=1. / 255, rotation_range=30, vertical_flip=True, horizontal_flip=True, zoom_range=[0.8, 1.2]
        ).flow_from_directory(train_path, target_size=(input_size, input_size), batch_size=BATCH_SIZE, classes=CLASSES)
    val_generator = ImageDataGenerator(rescale=1. / 255).flow_from_directory(
        val_path, target_size=(input_size, input_size), batch_size=BATCH_SIZE, classes=CLASSES)
    
    STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
    STEP_SIZE_VALID = val_generator.n // val_generator.batch_size
 
    model_history, base_model, model = model_fine_tuning(
        NET, base_model, model, train_generator, val_generator, EPOCHS_FINE_TUNING, STEP_SIZE_TRAIN,
        STEP_SIZE_VALID, BASE_RESULTS, str(fold) + '_pre')
    all_acc_histories.append(model_history.history['val_acc'])

print(all_acc_histories)
average_acc_history = [np.mean([x[i] for x in all_acc_histories]) for i in range(EPOCHS_FINE_TUNING)]
print(average_acc_history)
max_acc_epoch = average_acc_history.index(max(average_acc_history)) + 1
print('Best epoch for fine-tuning: {}'.format(max_acc_epoch))

all_acc_histories_deep = []
# For loop repeating the training for each fold: deep-tuning (train all layers)
for fold in range(1, NUM_FOLDS + 1):
    fold_path = BASE + '/fold_' + str(fold)
    train_path = fold_path + '/training_set'
    val_path = fold_path + '/validation_set'
    results_file = open(results_path + '/info.txt', 'w' if fold == 1 else 'a') 

    base_model, model = build_model(NET, input_size)
    train_generator = ImageDataGenerator(
        rescale=1. / 255, rotation_range=30, vertical_flip=True, horizontal_flip=True, zoom_range=[0.8, 1.2]
        ).flow_from_directory(train_path, target_size=(input_size, input_size), batch_size=BATCH_SIZE, classes=CLASSES)
    val_generator = ImageDataGenerator(rescale=1. / 255).flow_from_directory(
        val_path, target_size=(input_size, input_size), batch_size=BATCH_SIZE, classes=CLASSES)
    
    STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
    STEP_SIZE_VALID = val_generator.n // val_generator.batch_size
    
    # Do a fine-tuning first for the added top layers, using the best epoch of the fine-tuning performed previously
    model_history, base_model, model = model_fine_tuning(
        NET, base_model, model, train_generator, val_generator, max_acc_epoch,
        STEP_SIZE_TRAIN, STEP_SIZE_VALID, BASE_RESULTS, fold)
    # Deep-tuning: train all layers
    model_history = model_deep_tuning(
        NET, input_size, base_model, model, train_generator, train_generator, TEST_SET_PATH, EPOCHS_DEEP_TUNING,
        STEP_SIZE_TRAIN, STEP_SIZE_VALID, BASE_RESULTS, fold, results_file)
    all_acc_histories_deep.append(model_history.history['val_acc'])

print(all_acc_histories_deep)
average_acc_history_deep = [np.mean([x[i] for x in all_acc_histories_deep]) for i in range(EPOCHS_DEEP_TUNING)]
print(average_acc_history_deep)
max_acc_epoch_deep = average_acc_history_deep.index(max(average_acc_history_deep)) + 1
print('Best epoch for deep-tuning: {}'.format(max_acc_epoch_deep))

# Final training with all images using the best epoch found for fine and deep tuning
results_file = open(results_path + '/final_train_test.txt', 'w') 
results_file.write('Fine-tuning epochs: {}\n'.format(max_acc_epoch))
results_file.write('Deep-tuning epochs: {}\n'.format(max_acc_epoch_deep))
base_model, model = build_model(NET, input_size)
train_generator = ImageDataGenerator(
    rescale=1. / 255, rotation_range=30, vertical_flip=True, horizontal_flip=True, zoom_range=[0.8, 1.2]
    ).flow_from_directory(
        ALL_TRAIN_SET_PATH, target_size=(input_size, input_size), batch_size=BATCH_SIZE, classes=CLASSES)
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size

model_history, base_model, model = model_fine_tuning(NET, base_model, model, train_generator, None, max_acc_epoch,
                                                     STEP_SIZE_TRAIN, None, BASE_RESULTS, 'final')
model_history = model_deep_tuning(NET, input_size, base_model, model, train_generator, None, TEST_SET_PATH,
                                  max_acc_epoch_deep, STEP_SIZE_TRAIN, None, BASE_RESULTS, 'final', results_file)
